# Training a transformer model

In [1]:
checkpoint = 'distilbert-base-uncased'

## Loading Tokenizer

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

## Dataset Processing

### Dataset Loading

In [3]:
from datasets import load_dataset
from transformers import DataCollatorWithPadding

raw_dataset = load_dataset('financial_phrasebank', 'sentences_allagree')
raw_dataset = raw_dataset['train'].train_test_split(test_size=0.2, stratify_by_column="label")

def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)


tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1811 [00:00<?, ? examples/s]

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

In [4]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1811
    })
    test: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 453
    })
})

### Initialising Data Collator

In [5]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Preparing data for training

In [6]:
from torch.utils.data import DataLoader

In [7]:
tokenized_dataset = tokenized_dataset.remove_columns(["sentence"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")
tokenized_dataset["train"].column_names

['labels', 'input_ids', 'attention_mask']

In [8]:
train_dataloader = DataLoader(
                        tokenized_dataset["train"],
                        shuffle=True,
                        batch_size=16,
                        collate_fn=data_collator
                    )

In [9]:
test_dataloader = DataLoader(
                        tokenized_dataset["test"],
                        batch_size=16,
                        collate_fn=data_collator
                    )

## Model Training

### Loading Model

In [10]:
from transformers import AutoModelForSequenceClassification

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### Initialising Optimizer

In [12]:
from transformers import AdamW

In [13]:
optimizer = AdamW(model.parameters(), lr=3e-5)

/home/akshay/DLNLP/nlp_env/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Initialising Scheduler

In [14]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

342


### training Loop

In [15]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

device

device(type='cpu')

In [17]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        output = model(**batch)
        loss = output.loss
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/342 [00:00<?, ?it/s]

Inside the training loop, optimization happens in three steps:
* Backpropagate the prediction loss with a call to `loss.backward()`. PyTorch deposits the gradients of the loss w.r.t. each parameter.
* Once we have our gradients, we call `optimizer.step()` and `lr_sceduler.step()` to adjust the parameters by the gradients collected in the backward pass.
* Call `optimizer.zero_grad()` to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.

## Saving the model

In [18]:
model.save_pretrained('./../../../model/fin_sentiment_distilbert/')

## Model Evaluation

In [19]:
import numpy as np
from datasets import load_metric

load_accuracy = load_metric("accuracy")
load_f1 = load_metric("f1")
model = AutoModelForSequenceClassification.from_pretrained('./../../../model/fin_sentiment_distilbert/')

model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    load_accuracy.add_batch(predictions=predictions, references=batch["labels"])
    load_f1.add_batch(predictions=predictions, references=batch["labels"])

print(load_accuracy.compute())
print(load_f1.compute(average='macro'))

/tmp/ipykernel_21260/260688807.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'accuracy': 0.9690949227373068}
{'f1': 0.961432160269183}
